# Predicting the Status of Wells Across Tanzania

Exploration done by WellWatchers Avi Saraf, Lily Zhang, and Cindy Zhao.

In [1]:
import pandas as pd
import numpy as np

###Data preparation

In [2]:
df_values = pd.read_csv("clean_training_set_values.csv")
df_labels = pd.read_csv("training_set_labels.csv")

In [41]:
#merge the features (i.e. values) and the labels into one DataFrame
df = pd.merge(df_values, df_labels, on="id")

In [67]:
df.head()

id  amount_tsh  funder  gps_height installer  longitude   latitude  \
0  69572        6000   Other        1390     Other  34.938093  -9.856322   
1   8776           0   Other        1399     Other  34.698766  -2.147466   
2  34310          25   Other         686     Other  37.460664  -3.821329   
3  67743           0  Unicef         263     Other  38.486161 -11.155298   
4  19728           0   Other           0     Other  31.130847  -1.825359   

                     basin  subvillage   region       ...            quantity  \
0               Lake Nyasa    Mnyusi B   Iringa       ...              enough   
1            Lake Victoria     Nyamara     Mara       ...        insufficient   
2                  Pangani     Majengo  Manyara       ...              enough   
3  Ruvuma / Southern Coast  Mahakamani   Mtwara       ...                 dry   
4            Lake Victoria  Kyanyamisa   Kagera       ...            seasonal   

  quantity_group                source           source_type source_class  \
0         enough                spring                spring  groundwater   
1   insufficient  rainwater harvesting  rainwater harvesting      surface   
2         enough                   dam                   dam      surface   
3            dry           machine dbh              borehole  groundwater   
4       seasonal  rainwater harvesting  rainwater harvesting      surface   

               waterpoint_type waterpoint_type_group  \
0           communal standpipe    communal standpipe   
1           communal standpipe    communal standpipe   
2  communal standpipe multiple    communal standpipe   
3  communal standpipe multiple    communal standpipe   
4           communal standpipe    communal standpipe   

  date_recorded_offset_days date_recorded_month    status_group  
0                      1024                 Mar      functional  
1                       301                 Mar      functional  
2                       310                 Feb      functional  
3                       338                 Jan  non functional  
4                       903                 Jul      functional  

[5 rows x 34 columns]

#Categorical Features

We have the following categorical variables:
- funder
- installer
- basin
- subvillage
- region
- scheme_management
- management
- management_group
- public_meeting (T/F)
- permit (T/F)
- extraction_type
- extraction_type_group
- extraction_type_class
- payment
- payment_type
- water_quality
- quality_group
- waterpoint_type
- waterpoint_type_group

Here we see the unique values under the columns with categorical values.

##Redundancies
We see that some of the columns describe the same feature but have slightly different sets of values. This is particularly the case with features that also have a corresponding '_ _group_' column as well. For now, we will stick with the non-gropu features because they are more specific.

In [66]:
print pd.Series(df.extraction_type.ravel()).unique()
#less specific
print pd.Series(df.extraction_type_group.ravel()).unique()
print pd.Series(df.extraction_type_class.ravel()).unique()

['gravity' 'submersible' 'swn 80' 'nira/tanira' 'india mark ii' 'other'
 'ksb' 'mono' 'windmill' 'afridev' 'other - rope pump' 'india mark iii'
 'other - swn 81' 'other - play pump' 'cemo' 'climax' 'walimi']
['gravity' 'submersible' 'swn 80' 'nira/tanira' 'india mark ii' 'other'
 'mono' 'wind-powered' 'afridev' 'rope pump' 'india mark iii'
 'other handpump' 'other motorpump']
['gravity' 'submersible' 'handpump' 'other' 'motorpump' 'wind-powered'
 'rope pump']


In [65]:
#equivalent
print pd.Series(df.payment.ravel()).unique()
print pd.Series(df.payment_type.ravel()).unique()

['pay annually' 'never pay' 'pay per bucket' 'unknown'
 'pay when scheme fails' 'other' 'pay monthly']
['annually' 'never pay' 'per bucket' 'unknown' 'on failure' 'other'
 'monthly']


In [33]:
#the difference here is the addition of 'communal standpipe multiple' in waterpoint_type
print pd.Series(df.waterpoint_type.ravel()).unique()
#less specific
print pd.Series(df.waterpoint_type_group.ravel()).unique()

['communal standpipe' 'communal standpipe multiple' 'hand pump' 'other'
 'improved spring' 'cattle trough' 'dam']
['communal standpipe' 'hand pump' 'other' 'improved spring' 'cattle trough'
 'dam']


In [31]:
print pd.Series(df.water_quality.ravel()).unique()
#less specific
print pd.Series(df.quality_group.ravel()).unique()

['soft' 'salty' 'milky' 'unknown' 'fluoride' 'coloured' 'salty abandoned'
 'fluoride abandoned']
['good' 'salty' 'milky' 'unknown' 'fluoride' 'colored']


In [32]:
#less specific
#'borehole' includes 'machine dbh' and 'hand dtw'
#'river/lake' includes 'river' and 'lake'
#'other' includes 'other' and 'unknown'
print pd.Series(df.source_type.ravel()).unique()
print pd.Series(df.source.values.ravel()).unique()
#source and source_type are nested within source_class
print pd.Series(df.source_class.values.ravel()).unique()

['spring' 'rainwater harvesting' 'dam' 'borehole' 'other' 'shallow well'
 'river/lake']
['spring' 'rainwater harvesting' 'dam' 'machine dbh' 'other' 'shallow well'
 'river' 'hand dtw' 'lake' 'unknown']
['groundwater' 'surface' 'unknown']


In [43]:
#management is nested within management_group
print pd.Series(df.management.ravel()).unique()
print pd.Series(df.management_group.ravel()).unique()

['vwc' 'wug' 'other' 'private operator' 'water board' 'wua' 'company'
 'water authority' 'parastatal' 'unknown' 'other - school' 'trust']
['user-group' 'other' 'commercial' 'parastatal' 'unknown']


We see that some of these features are redundant, so we delete one of the columns of these repeats. Regarding the features/feature_group label pairs, there is a nesting structure but it is so slight that we just delete one of the columns; otherwise, we would most likely run into problems of collinearity. We have two new dataframes, one with only the feature_group labels (less specific), and one with only the feature labels (more specific). We will try our regression on the more specific df_new but keep df_new1 just in case.

In [70]:
df_new = df.drop(['quantity_group','extraction_type_group','waterpoint_type_group','quality_group','source_type','payment'], axis=1)
df_new1 = df.drop(['quantity_group','extraction_type','waterpoint_type','quality_group','source','payment'], axis=1)

##Nestings

In df_new, we know that:
- management is nested within management_group
- source is nested within source_class
- extraction is nested within extraction_type_class
- subvillage is nested within region

##Grouping the types of categorical variables
The features dealing with the management of the well include:
- funder
- installer
- scheme_management
- management/management_group
- payment_type
- permit (T/F)

The features dealing with the creation of the well include:
- extraction/extraction_type_class
- waterpoint_type

The features dealing with time include: 
- construction_year
- date_recorded_offset_days (int; how long ago it was constructed, from the date recorded)
- date_recorded_month

The features dealing with natural properties include:
- water_quality
- source/source_class
- amount_tsh
- quantity
- basin

The features dealing with the community using the well and location include:
- subvillage
- region
- population
- public_meeting (T/F)
- gps_height
- longitude
- latitude

In [76]:
df_new.columns

Index([u'id', u'amount_tsh', u'funder', u'gps_height', u'installer',
       u'longitude', u'latitude', u'basin', u'subvillage', u'region',
       u'population', u'public_meeting', u'scheme_management', u'permit',
       u'construction_year', u'extraction_type', u'extraction_type_class',
       u'management', u'management_group', u'payment_type', u'water_quality',
       u'quantity', u'source', u'source_class', u'waterpoint_type',
       u'date_recorded_offset_days', u'date_recorded_month', u'status_group'],
      dtype='object')